In [12]:
import pandas as pd
from xgboost import XGBRegressor
import numpy as np

# Load your dataset
df = pd.read_csv('/content/NumreicalValues_Season3 - Sheet1.csv')



In [13]:
missing_columns = df.columns[df.isnull().any()].tolist()

# Imputation Loop
for col in missing_columns:
    # Create training and testing data for this column
    X_train = df.drop(columns=[col])
    y_train = df[col].copy()

    # Drop rows with missing values in both features and target
    missing_rows = y_train.isnull()
    X_train = X_train[~missing_rows]
    y_train = y_train.dropna()


    X_train = X_train.fillna(-9999)


    if y_train.isnull().any() or not np.isfinite(y_train).all():
        print(f"Skipping column '{col}' due to NaNs, infinite, or too large values in the target labels.")
        continue

    # Train XGBoost model
    model = XGBRegressor()
    model.fit(X_train, y_train)

    # Impute missing values
    X_missing = df[df[col].isnull()]
    if not X_missing.empty:
        imputed_values = model.predict(X_missing.drop(columns=[col]))

        # Update missing values in the original DataFrame
        df.loc[df[col].isnull(), col] = imputed_values


df.to_csv('XGBoost_Season3.csv', index=False)